In [1]:
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service


In [2]:
href_list = []
all_Home_List = []

def listToString(s):
    str1 = ""
    return (str1.join(s))

def strRegex(str):
    return str[0].replace("M2","").replace("m2","").replace(". Kat","").replace("TL","").replace("Yaşında","").replace("Katlı","").strip()

# HepsiEmlak Bot

In [9]:
#https://www.hepsiemlak.com/istanbul-kiralik/daire?sortField=PRICE&sortDirection=DESC&p32=40000&p33=1

#link topla
s = Service("C:\DataMining\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(service=s)
driver.maximize_window()

for page_num in range(1,2):
    
    driver.get("https://www.hepsiemlak.com/istanbul-kiralik/daire?p31=3000&p32=45000&p33=1&page=" + str(page_num) + "")

    # ilan
    liste_item = driver.find_elements(by=By.CSS_SELECTOR, value="div[class='list-view-content']>div[class='links']>a")

    for i in range(len(liste_item)):
        href_list.append(liste_item[i].get_attribute('href'))

In [ ]:
        
#link gez 
for href_num in range(len(href_list)):
    time.sleep(1)   
    try:
        driver.get(href_list[href_num])
        
        # ilan içeriği
        pull_elements = driver.find_elements(by=By.CLASS_NAME, value="spec-groups")  # Teknik Özellik
        pull_price = driver.find_element(by=By.CLASS_NAME, value="price")  # fiyat
        ilce = driver.find_element(by=By.CSS_SELECTOR, value="#__layout > div > div > section.wrapper.detail-page > div > div.det-content.cont-block.left > div.cont-inner > section:nth-child(1) > div.det-title-bottom > ul > li:nth-child(2)")

        # özellikler
        list_elements = []
        for element in pull_elements:
            list_elements.append(element.text)

        # düzenle
        list_elements_str = listToString(list_elements)
        elements_splitter = list_elements_str.split("\n")
        df = pd.DataFrame(elements_splitter)

        df = df.iloc[8:31]
        df = df.reset_index()  # değişiklik index kayma fix
        df.drop("index", axis=1, inplace=True)  # oluşan index kaldır

        
        df = df.drop([15, 16, 17, 18, 19, 20])
        df = df.reset_index()  # değişiklik index kayma fix
        df.drop("index", axis=1, inplace=True)  # oluşan index kaldır
        print(df)
        
        
        # özellikler
        df_list = df.values.tolist()
        list_home_feature = []
        counter = len(df_list)
        for i in range(5):
            if (i % 2 == 1):
                list_home_feature.append(strRegex(df_list[i]))
            if (i == 4):
                val = strRegex(df_list[i])
                list_home_feature.append(val.replace("/ ","").strip()) 

        for i in range(5,len(df_list)):
            if (i % 2 == 0):
                list_home_feature.append(strRegex(df_list[i]))

        list_home_feature.append(ilce.text.strip()) # ilce add

        home_rent = pull_price.text.replace(".", ",")
        list_home_feature.append(home_rent[0:home_rent.index("TL")].strip())  # kira val add

        all_Home_List.append(list_home_feature)
    except:
        print("!!! Excepiton Page: " + str(href_num) )

#save to csv
df_home_features = pd.DataFrame(all_Home_List)

df_home_features.to_csv("hepsiEmlak.csv",
                        encoding="utf-8", index=False)

driver.quit()
print('-- ok')

                     0
0   Oda + Salon Sayısı
1                3 + 1
2        Brüt / Net M2
3               185 m2
4             / 140 m2
5        Bulunduğu Kat
6               9. Kat
7            Bina Yaşı
8            6 Yaşında
9          Isınma Tipi
10               Kombi
11          Kat Sayısı
12             9 Katlı
13        Banyo Sayısı
14                   3
15               Cephe
16               Güney
                      0
0    Oda + Salon Sayısı
1                 3 + 1
2         Brüt / Net M2
3                189 m2
4              / 130 m2
5         Bulunduğu Kat
6                5. Kat
7             Bina Yaşı
8             4 Yaşında
9           Isınma Tipi
10  Merkezi (Pay Ölçer)
11           Kat Sayısı
12             23 Katlı
13          Eşya Durumu
14               Eşyalı
15      Kullanım Durumu
16                  Boş
                      0
0    Oda + Salon Sayısı
1                 1 + 1
2         Brüt / Net M2
3                 70 m2
4               / 50 m2
5         

In [3]:
df_excel=pd.read_csv("hepsiEmlak.csv")
df_excel

,0,1,2,3,4,5,6,7,8,9,10
0,1 + 1,70.0,50,21 ve üzeri,5,Merkezi (Pay Ölçer),30,Eşyalı,Boş,Sarıyer,"25,000"
1,2 + 1,109.0,80,5,Sıfır Bina,Merkezi (Pay Ölçer),14,Eşyalı,Kiracılı,Sarıyer,"35,000"
2,3 + 1,189.0,130,5,4,Merkezi (Pay Ölçer),23,Eşyalı,Boş,Eyüpsultan,"35,000"
3,3 + 1,140.0,120,En Üst Kat,40,Merkezi (Pay Ölçer),3,Eşyalı Değil,Boş,Kadıköy,"14,000"
4,4 + 1,220.0,180,Bahçe Katı,10,Kombi,4,Eşyalı Değil,Boş,Sarıyer,"40,000"
...,...,...,...,...,...,...,...,...,...,...,...
5911,3 + 1,170.0,120,3,2,Merkezi,9,Eşyalı Değil,Boş,Eyüpsultan,"40,000"
5912,2 + 1,100.0,67,21 ve üzeri,3,Merkezi,22,Eşyalı Değil,Boş,Eyüpsultan,"35,000"
5913,3 + 1,170.0,130,6,9,Merkezi (Pay Ölçer),33,Eşyalı Değil,Boş,Sarıyer,"35,000"
5914,3 + 1,165.0,135,5,Sıfır Bina,Merkezi (Pay Ölçer),20,Eşyalı Değil,Boş,Sarıyer,"38,000"
